In [ ]:
from datetime import timedelta
from pathlib import Path

from flow_analysis_comps.data_structs import (
    videoInfo,
    GST_params,
    kymoExtractConfig,
    kymoOutputs
)
import tifffile
from flow_analysis_comps.processing.kymographing.kymographer import KymographExtractor
from flow_analysis_comps.scripts.classic_flow_extraction import (
    process_kymo,
    process_video,
)

%matplotlib widget

%load_ext autoreload
%autoreload 2

# Running the classic extraction: Whole videos
Below, a whole video will be processed using the input arguments that you can give in the metadata. In the first block you input the video settings, then in the second block you can run the analysis.

In [ ]:
from flow_analysis_comps.data_structs.video_metadata_structs import cameraPosition, cameraSettings, videoMode


Input_video_path = Path(r"/Users/simonvanstaalduine/AMOLF-SHIMIZU Dropbox/Simon van Staalduine/134438")
Output_folder = Path(r"/Users/simonvanstaalduine/AMOLF-SHIMIZU Dropbox/Simon van Staalduine/Output")
frames_paths_list = list(Input_video_path.glob("*.ti*"))

camera_settings = cameraSettings(
    model="Davis",
    exposure_us=100.0,
    frame_rate=30.0,
    frame_size=[2040, 520],  # Pixel size of single image
    binning=2,
    gain=1.0,
    gamma=0.0,
    pixel_size=1.725,  # um camera sensor pixel size
)

input_metadata = videoInfo(
    root_path=Input_video_path,
    duration=timedelta(seconds=10),
    frame_nr=len(frames_paths_list),
    mode=videoMode.BRIGHTFIELD,  # BRIGHTFIELD or FLUORESCENCE
    magnification=50.0,
    camera=camera_settings,
    position=cameraPosition(x=0.0, y=0.0, z=0.0),  # Position of camera in micrometers
)
speed_config = GST_params()
graph_extraction_config = kymoExtractConfig(target_length=80)  # Pixel width of hyphae

data_output_folder = Output_folder / Input_video_path.name
data_output_folder.mkdir(exist_ok=True)


In [ ]:
process_video(
    Input_video_path,
    data_output_folder,
    speed_config,
    graph_extraction_config,
    user_metadata=input_metadata,
)

# Running the classic extraction: Single kymographs
If you have a single kymograph, and you want to process it, you need to give the dx and dt, and the rest will be sorted for you.

In [ ]:
input_img_path = Path(r"U:\test_data\temp_flow_output\kymo_folder\SCP_test.tif")
out_folder = Path(r"U:\test_data\temp_flow_output\kymo_folder")
out_data_folder = (out_folder / input_img_path.stem).mkdir(exist_ok=True)
input_img = tifffile.imread(input_img_path)
input_kymos = KymographExtractor._decompose_kymograph(input_img)

frame_rate = 20.0  # Hz
camera_sensor_size_um = 1.725  # um camera sensor pixel size
camera_binning = 2  # Binning factor for the camera
magnification = 50.0  # Magnification of the microscope

kymograph = kymoOutputs(
    deltas=kymoDeltas(
        delta_t=1 / frame_rate,
        delta_x=camera_sensor_size_um / magnification * camera_binning * 2,
    ),
    name="SCP_test",
    kymograph=input_img,
    kymo_left=input_kymos[0],
    kymo_right=input_kymos[1],
    kymo_no_static=input_kymos[0] + input_kymos[1],
)

In [ ]:
process_kymo(
    kymograph, out_folder, speed_config
)

In [ ]:
"""Old code that has been replaced by the process_video function.
This code is kept for reference and should not be executed."""

# graph_data = VideoGraphExtractor(
#     Input_video_path, graphExtractConfig(), user_metadata=input_metadata
# ).edge_data
# kymograph_list = KymographExtractor(
#     graph_data, graph_extraction_config
# ).processed_kymographs
# edge_extraction_fig = GraphVisualizer(
#     graph_data, graph_extraction_config
# ).plot_extraction()
# edge_extraction_fig.savefig(data_output_folder / "edges_map.png")

# for kymo in kymograph_list:
#     kymo_speeds = kymoAnalyser(kymo, speed_config).output_speeds()
#     edge_out_folder = data_output_folder / f"{kymo.name}"
#     edge_out_folder.mkdir(exist_ok=True)
#     analyser = kymoAnalyser(kymo, speed_config)
#     fig, ax = GSTSpeedVizualizer(kymo_speeds).plot_summary(kymo)
#     fig.savefig(edge_out_folder / f"{input_metadata.position}_{kymo.name}_summary.png")
#     time_series, averages = analyser.return_summary_frames()
#     time_series.to_csv(edge_out_folder / f"{kymo.name}_time_series.csv")
#     averages.to_csv(edge_out_folder / f"{kymo.name}_averages.csv")

In [ ]:
"""Old code that has been replaced by the process_kymo function.
This code is kept for reference and should not be executed."""

# kymo_speeds = kymoAnalyser(kymograph, speed_config).output_speeds()
# edge_out_folder = out_folder / f"{kymograph.name}"
# edge_out_folder.mkdir(exist_ok=True)
# analyser = kymoAnalyser(kymograph, speed_config)
# fig, ax = GSTSpeedVizualizer(kymo_speeds).plot_summary(kymograph)
# fig.savefig(edge_out_folder / f"{kymograph.name}_summary.png")
# time_series, averages = analyser.return_summary_frames()
# time_series.to_csv(edge_out_folder / f"{kymograph.name}_time_series.csv")
# averages.to_csv(edge_out_folder / f"{kymograph.name}_averages.csv")